<h1>🤖 MLAI Workshop #02</h1>

In the previous workshop, we explored a foundational drive behind machine learning — thinking of it not just as code or models, but as a process of learning functions under constraints such as hypothesis, data and optimization. We emphasized the importance of the hypothesis space, how data shapes learning, and where errors can arise even when everything seems to be working correctly. Today, we’ll start to address some limitations of our prior models and try to interpret the limitations in the context of the theory - and consequently, what we can do to address it.

💬 *Question for the audience! Have you ever trained a model that just... wouldn’t learn — and you weren’t sure why? What do you think the cause of failure was?*

<h2>🗓️ Agenda</h2>

1. **Recap & Refactor**  
   Summarize key concepts from the previous workshop and refine our implementation using PyTorch's autograd and optimizers.

2. **Flexible Function Approximators**  
   Explore more expressive hypothesis spaces (e.g. neural networks) and understand their strengths and limitations in practice.

3. **Modelling in Context**  
   Apply these ideas to your own modelling task — build, train, and interpret a model on our own functions.

4. **Generalization & Limits**  
   Investigate how models behave beyond training data. Where do they succeed? Where do they fail? How can we improve them?


<h2>Summary</h2>

1. Machine learning is the task of approximating an unknown function using data.
2. Hypothesis Space: We define a family of functions (e.g. linear models, neural networks) that we’re willing to consider - this results in approximation error.
3. Data as Constraints: Each data point reduces the set of plausible functions by adding constraints - this results in generalization error.
4. Optimization: We use methods like gradient descent to minimize loss and fit the data - this results in optimization error.


---

<h2>📈 Section 1: What's wrong with our hypothesis space?</h2>

<h3>Section 1A. Revisiting Workshop #1</h3>

So we suppose we have some target function we want to approximate.

\begin{align*}
  f^{*}(x) = \left\{ \mathbb{R} \rightarrow \mathbb{R} \,|\, f(x) = -0.86 x + 1.4 \right\} \tag{1.1}
\end{align*}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# target function
def f_target(x):
    return -0.68 * x + 1.4

We assume we cannot access this function, but we can observe its behaviour - and thus we decide to collect a dataset that hopefully captures this functions behaviour.

\begin{align*}
    \mathcal{D} = \left\{ (\mathcal{X}_{i}, \mathcal{Y}_{i}) \right\}_{i=0}^{N} \tag{1.2}
\end{align*}

where:
- $\mathcal{X}_{i}$ represents an observation of the input.
- $\mathcal{Y}_{i}$ represents an observation of the output.

In [ ]:
# observe some data using a noisy observation process
def observe_noisy(f, x):
    # we often can't sample perfectly a point in the domain perfectly
    x_noise = np.random.normal(0.05, 0.02, x.shape[0])
    x_measure = x + x_noise

    # we often can't measure the result perfectly
    y_noise = np.random.normal(-0.05, 0.15, x.shape[0])
    y_obs = f(x_measure) + y_noise

    return y_obs

# we select some points to observe the response at
x_meas = np.random.uniform(low=-1, high=1, size=100)
y_obs = observe_noisy(f_target, x_meas)

# plot the observations
fig, ax = plt.subplots(figsize=(8,3))
ax.scatter(x_meas, y_obs, s=6, label="observations")
ax.set_xlim(x_meas.min(), x_meas.max())
ax.legend(loc="best")
ax.set_xlabel("x_meas")
ax.set_ylabel("y_obs")

After some exploratory data analysis we might hypothesize this function has a linear relationship, and we design a very specific constricted hypothesis space to match this:

\begin{align*}
    \mathcal{H} = \left\{ f_{\theta}: \mathbb{R} \to \mathbb{R} \,|\, f_{\theta}(x) = \theta_{1} x + \theta_{2} \right\} \tag{1.3}
\end{align*}


In [ ]:
# construct a hypothesis space
def H(a, b):
    def f(x):
        return a * x + b
    return f

To provide an empirical evaluation of the fit of our function we define a loss function that meaningfully evaluates the task we want our model to perform:

\begin{align*}
    \mathcal{L}(\mathcal{D}, f_\theta) = \frac{1}{n} \sum_{i=1}^{n} \ell\left(\hat{y}_{i}, y_i\right) = \frac{1}{n} \sum_{i=1}^{n} \ell\left(f_\theta(x_i), y_i\right) \tag{1.4}
\end{align*}

In [ ]:
# define a loss function
def dataset_avg_mse_loss(f, x, y):
    # compute average mse loss between prediction `f(x)` and observation `y`
    L = np.sum((f(x) - y) ** 2) / x.shape[0]
    return L

# evaluate a specific model
L_mse = dataset_avg_mse_loss(H(1,1), x_meas, y_obs)
L_mse

We then also define the target spaces as a constraint on our set of approximations we consider useful:

\begin{align*}
    \mathcal{T} = \left\{ f \in \mathcal{H} \,|\, \mathcal{L}(\mathcal{D}, f_{\theta} \leq \epsilon) \right\} \tag{1.5}
\end{align*}

In [ ]:
# function to check if the solution is suitable
def is_suitable(L_mse, threshold):
    return L_mse < threshold

Since we have a simple model we can visualize the loss landscape in a meaningful manner before we begin the optimization process.

In [ ]:
# visualize the loss landscape for understanding
a = np.linspace(-5, 5, num=100) # <-- explore a different range, does [-5,5] let you find the optimal solution?
b = np.linspace(-5, 5, num=100)
A, B = np.meshgrid(a, b, indexing="ij")

# lets compute the loss (error across the dataset) for each function defined by the parameter
losses = np.zeros_like(A)
for i in range(a.shape[0]):
    for j in range(b.shape[0]):
        # select a function from the hypothesis space
        f_approx = H(a=a[i], b=b[j])

        # compute loss across dataset
        losses[i,j] = dataset_avg_mse_loss(f_approx, x_meas, y_obs)

# lets plot the result
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection="3d")
s = ax.plot_surface(A, B, losses, cmap='viridis')
ax.set_xlim(a.min(), a.max())
ax.set_ylim(b.min(), b.max())
ax.set_xlabel("a")
ax.set_ylabel("b")
ax.set_zlabel("Dataset Loss (MSE)")
ax.set_title(f"Loss Landscape")

We can define the gradients of our loss function with respect to each parameter of our model, which we then use in the gradient descent algorithm.

\begin{align*}
\theta := \theta - \eta \cdot \nabla_\theta \mathcal{L}(f_\theta(x), y) \tag{1.6}
\end{align*}

In [ ]:
# derivate of function with respect to parameters
def df_da(x, a, b):
    return x

def df_db(x, a, b):
    return 1

# derivate of loss function with respect to params : chain rule
def dL_dtheta(x, y, a, b):
    dL_da = 2 * np.mean((H(a, b)(x) - y) * df_da(x, a, b))
    dL_db = 2 * np.mean((H(a, b)(x) - y) * df_db(x, a, b))
    return (dL_da, dL_db)

We start our optimization process at some initial point on the loss landscape $(\theta_{0}, \theta_{1})$, using the gradient information we then update the parameter values by $\nabla_\theta \mathcal{L}(f_\theta(x), y)$ where $\nabla_\theta$ represents a scaling of the gradients. We repeat this process for $N$ steps.

In [ ]:
# optimization using gradient descent
def gradient_descent(a_init, b_init, x, y, lr, steps):
    a, b = a_init, b_init
    history = []

    for idx in range(steps):
        # compute gradients
        dL_da, dL_db = dL_dtheta(x, y, a, b)

        # curr loss
        L_mse = dataset_avg_mse_loss(H(a, b), x, y)
        history.append((a, b, L_mse))

        # gradient descent update step
        a = a - lr * dL_da
        b = b - lr * dL_db

    return np.array(history)

# run gradient descent and get final parameters
trajectory = gradient_descent(a_init=.0, b_init=.0, x=x_meas, y=y_obs, lr=0.1, steps=100, )
f_approx = H(trajectory[-1,0], trajectory[-1,1])

# exit condition : found suitable solution
if is_suitable(trajectory[-1,-1], 0.01): print(f"found suitable solution with loss L={trajectory[-1,-1]:.3f}")

In [ ]:
# Plot loss over iterations
fig, ax = plt.subplots(figsize=(8,3))
ax.plot(trajectory[:, 2], label="L_mse")
ax.set_xlim(left=0, right=len(trajectory)-1)
ax.set_xlabel("Iteration")
ax.set_ylabel("Loss (MSE)")
ax.set_title("Loss Curve")
ax.grid(True)
ax.legend()

# Plot the path on the loss landscape
fig, ax = plt.subplots(figsize=(8, 8))
contour = ax.contourf(A, B, losses, levels=100, cmap='viridis')
ax.set_xlim(A.min(), A.max())
ax.set_ylim(B.min(), B.max())
ax.set_xlabel("a")
ax.set_ylabel("b")
ax.set_title("Gradient Descent Path on Loss Landscape")

# Path
ax.plot(trajectory[0, 0], trajectory[0, 1], 'rx', label="Initial Point")
ax.plot(trajectory[:, 0], trajectory[:, 1], 'r-', alpha=0.25, label="Gradient Descent Path")
ax.plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', label="Final Point")
ax.legend()

# Plot predictions
fig, ax = plt.subplots(figsize=(8, 3))
ax.scatter(x_meas, f_approx(x_meas), s=6, label="predictions")
ax.scatter(x_meas, y_obs, s=6, label="observations")
ax.legend(loc="best")

<h3>Section 1B. Refining Our Code</h3>

Building everything from scratch helps us understand the mechanics of learning, but it quickly becomes tedious and error-prone as our models grow in complexity. Thankfully, libraries such as `torch` come with all of the functionality we need out of the box e.g the `autograd` engine to handle gradient computation.

Let's convert this code over into a `torch` style implementation.

In [ ]:
import torch
import torch.nn as nn
from typing import Callable

class Dataset:
    def __init__(self, f: Callable, a: float, b: float, N = 100):
        self.__f = f
        self.x = torch.from_numpy(np.random.uniform(low=a, high=b, size=N)).to(dtype=torch.float32)
        self.y = torch.from_numpy(observe_noisy(self.__f, self.x.numpy())).to(dtype=torch.float32)
        self.x = self.x.unsqueeze(-1) # [N,1]
        self.y = self.y.unsqueeze(-1) # [N,1]

        # <-- define some additional processing here if you want

In [ ]:
import torch
import torch.nn as nn

class HypothesisSpace(nn.Module):
    """ implement a simple linear equation hypothesis space
    f(x) = a * x + b
    """
    def __init__(self, a: float, b: float):
        super().__init__()
        self.a = nn.Parameter(torch.tensor([a]))
        self.b = nn.Parameter(torch.tensor([b]))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.a * x + self.b

In [ ]:
from tqdm import tqdm

# define the optimization loop aka. training loop
def training_loop(model, optimizer, loss_fn, x, y, steps):
    losses = []
    with tqdm(range(steps)) as pbar:
      for idx in pbar:
          optimizer.zero_grad()
          loss = loss_fn(y, model(x))
          losses.append(loss)
          loss.backward()
          optimizer.step()
          pbar.set_description(f"loss: {loss.item():.3f}")
    return model, torch.tensor(losses)

In [ ]:
# define a dataset
dataset = Dataset(f=f_target, a=-2, b=2, N=100)

# initial model
model = HypothesisSpace(1.0, 1.0)

# define optimizer : autograd handles computing gradients etc.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# define loss function
loss_fn = torch.nn.MSELoss()

# perform the training loop
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)
if is_suitable(losses[-1], 0.01): print(f"found suitable solution with loss L={losses[-1]:.3f}")

In [ ]:
def plot_loss_curve(losses):
    fig, ax = plt.subplots(figsize=(8,3))
    ax.plot(losses, label="loss")
    ax.set_xlim(left=0, right=len(losses)-1)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss (MSE)")
    ax.set_title("Loss Curve")
    ax.grid(True, alpha=0.50)
    ax.legend()
    return fig, ax

def plot_predictions(model, dataset, ax = None):
    if ax is None: fig, ax = plt.subplots(figsize=(8,3))
    with torch.no_grad():
        ax.scatter(dataset.x, model(dataset.x), s=4, label="predictions")
        ax.scatter(dataset.x, dataset.y, s=4, label="observations")
    ax.set_xlim(left=dataset.x.min(), right=dataset.x.max())
    ax.grid(True, alpha=0.50)
    ax.legend(loc="best")
    return ax

plot_loss_curve(losses)
plot_predictions(model, dataset)

<h3>Section 1C. What happens when our function gets more complicated?</h3>

So with this engineering issue sorted, lets begin to explore what happens when our target function becomes more complicated. Consider we have some slighty more complicated target function:

\begin{align*}
  f^{*}(x) = \left\{ \mathbb{R} \rightarrow \mathbb{R} \,|\, f(x) = 1.54x^{2} - 0.54x + 1.3 \right\} \tag{1.7}
\end{align*}

In [ ]:
# define a dataset
f_quadratic = lambda x: 1.54 * x**2 - 0.54 * x + 1.3
dataset = Dataset(f=f_quadratic, a=-2, b=2, N=100)

# visualize the observations
fig, ax = plt.subplots(figsize=(8,3))
ax.scatter(dataset.x, dataset.y, s=6, label="observations")
ax.set_xlim(dataset.x.min(), dataset.x.max())
ax.legend(loc="best")
ax.set_xlabel("x_meas")
ax.set_ylabel("y_obs")

In [ ]:
class HypothesisSpace(nn.Module):
    def __init__(self, a: float, b: float): # <-- add some inputs
        super().__init__()
        self.a = nn.Parameter(torch.tensor([a])) # <-- add some extra parameters
        self.b = nn.Parameter(torch.tensor([b]))

    def forward(self, x: torch.Tensor) -> torch.Tensor: # <-- rewrite the hypothesis space
        return self.a * x + self.b

In [ ]:
# initial model
model = HypothesisSpace(0.0, 0.0)

# define optimizer : autograd handles computing gradients etc.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# perform the training loop
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

# plot loss curve & predictions
plot_loss_curve(losses)
plot_predictions(model, dataset)

We assume in this scenario that both the optimization process and the dataset are not limiting factors — the model is training properly, and the data is representative. Yet, we still fail to achieve satisfactory performance. In this case, the limitation lies in the **approximation error**: the best possible function within our hypothesis space is still far from the true underlying function. This occurs because our hypothesis space is too restrictive. For instance, a linear model cannot effectively approximate a cubic polynomial — the complexity of the true function exceeds the representational capacity of the model.

This is a key insight: even perfect optimization can't help if we're searching within the wrong space. To improve performance, we must **redefine our hypothesis space** — expand it to include more expressive functions that can better capture the structure in the data.

👉 **Try this yourself:** Modify your model to use a more flexible hypothesis space (e.g. a higher-degree polynomial) and observe how performance changes.

---

<h2>Section 2. Universal Function Approximators</h2>

Clearly, the process of repeatedly modifying our hypothesis space isn’t a scalable strategy for learning highly complex functions. In many real-world tasks, the true relationship between input $\mathcal{X}$ and output $\mathcal{Y}$ is unknown, nonlinear, and potentially very intricate. We rarely have a closed-form expression for this mapping — and even if it exists, we typically cannot guess its structure reliably.

To address this challenge, we turn to more **flexible function classes** that are capable of approximating a wide variety of functions. Conceptually, this means expanding our hypothesis space $\mathcal{H}$ so that it can make more effective use of the parameter space $\Theta$, and better fit the patterns within the data.

<h3>Section 2A. Universal Function Approximation</h3>

What we seek is a **universal function approximator**: a model that, under certain conditions, can approximate *any* continuous function on a bounded domain to *arbitrary accuracy*, given sufficient capacity — for example, enough neurons, basis functions, or ensemble components.

Examples of such expressive hypothesis spaces include:
- **Multilayer perceptrons (neural networks)**, which construct complex nonlinear mappings by composing layers of simple transformations (e.g., linear projections + nonlinear activations).
- **Kernel methods** with radial basis functions (RBF), which implicitly map data into infinite-dimensional feature spaces to model complex, nonlinear relationships.

These flexible spaces are what empower modern machine learning systems to perform across a wide range of tasks — from image recognition to natural language processing — without the need to hand-design the function form in advance.

> Universal function approximators shift the burden of function design from the human to the learning algorithm.

Let’s now look at one such model in detail and explore how it achieves this.



<h3>Section 2B. Linear Transformation with Recitified Linear Units (ReLU)</h3>

\begin{align*}
  f(x) = ReLU(w_i x + b_i) \tag{2.1}
\end{align*}

We've seen what linear transformation does - what about this ReLU component? Each ReLU unit introduces a kink — a non-smooth point — at the location where its input changes sign. For a neuron computing $\text{ReLU}(w_i x + b_i)$, this kink occurs at the hyperplane $w_i x + b_i = 0$.

\begin{align*}
  \text{ReLU}(x) = \max(0, x) \tag{2.2}
\end{align*}

This divides the input space into **two linear regions**:
- One where the neuron is active ($w_i x + b_i > 0$)
- One where it is inactive ($w_i x + b_i \leq 0$)

Lets visualize this.


In [ ]:
class Perceptron(nn.Module):
    """
    f(x) = ReLU(Wx + B)
    """
    def __init__(self, w: float, b: float):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        self.linear.weight = nn.Parameter(torch.tensor([[float(w)]]))
        self.linear.bias = nn.Parameter(torch.tensor([[float(b)]]))
        self.act = nn.ReLU() # <-- try different activation functions e.g. Tanh, SiLU, GeLU

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.act(self.linear(x))

with torch.no_grad():
    x = torch.linspace(-5, 5, 1000).unsqueeze(-1)
    y = Perceptron(1.0, 0.0)(x) # <-- vary the parameters of the perceptron

    # plot the perceptron function
    fig, ax = plt.subplots(figsize=(8,3))
    ax.plot(x,y)
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(-2, 5)
    ax.set_title("ReLU Perceptron")
    ax.set_xlabel("x")
    ax.set_ylabel("f(x) = ReLU(wx + b)")

We can explore optimizing a single perceptron for our scenario.

In [ ]:
class Perceptron(nn.Module):
    """
    f(x) = ReLU(Wx + B)
    """
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

        # specific initialization
        self.linear.weight = nn.Parameter(torch.tensor([[1.0]])) # <-- define a specific starting model init
        self.linear.bias = nn.Parameter(torch.tensor([[0.0]]))

        # random initialization
        # nn.init.uniform_(self.linear.weight, -1, 1)
        # nn.init.uniform_(self.linear.bias, -1, 1)

        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.act(self.linear(x))


# define the model, dataset, optimizer, and train the model
model = Perceptron()
dataset = Dataset(f=f_quadratic, a=-2, b=2, N=100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

# visualize the losses and predictions of the trained model
plot_loss_curve(losses)
plot_predictions(model, dataset)

<h3>Section 2C. Single Layer Perceptron Model</h3>

Obviously, a single perceptron is quite limited in what it can approximate — it can only model simple linear relationships within a region defined by its activation boundary, or **kink**, in the input space.

However, when we introduce **multiple perceptron units**, each with its own activation boundary, these kinks collectively partition the input domain into distinct regions. Within each region, the network behaves linearly — but across regions, it can express **nonlinear behavior** by stitching together these local linear segments.

This gives rise to a **piecewise linear function**: a continuous function composed of linear components. In the **theoretical limit**, a **single-hidden-layer ReLU network with infinite width** becomes a **universal function approximator**. Under suitable conditions (such as proper weight initialization and scaling), it can approximate any continuous function on a compact input domain to arbitrary accuracy.

We’ll now explore how this works in practice.


In [ ]:
class PerceptronLayer(nn.Module):
    """
    f(x) = Perceptron0(x) + ... + PerceptronN(x)
    """
    def __init__(self, N: int = 1):
        super().__init__()
        self.perceptrons = nn.ModuleList([Perceptron() for _ in range(N)])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_pred = 0
        for perceptron in self.perceptrons:
            y_pred = y_pred + perceptron(x)
        return y_pred


model = PerceptronLayer(N=2) # <-- lets define 2 perceptrons initially
dataset = Dataset(f=f_quadratic, a=-2, b=2, N=100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

plot_loss_curve(losses)
plot_predictions(model, dataset)

Does this align with what you expected? What do you think is the issue?

In [ ]:
with torch.no_grad():
    x = torch.linspace(-5, 5, 1000).unsqueeze(-1)

    # plot the perceptron function
    fig, ax = plt.subplots(figsize=(8,3))

    for idx, perceptron in enumerate(model.perceptrons):
        ax.plot(x, perceptron(x), label=f"perceptron {idx}", alpha=0.75)

    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(-2, 5)
    ax.set_title("ReLU Perceptron")
    ax.set_xlabel("x")
    ax.set_ylabel("f(x) = ReLU(wx + b)")
    ax.legend(loc="best")
    ax.grid(True, alpha=0.50)

Initialization plays a critical role in neural network training. If all weights are initialized to the same value, the network becomes symmetric — meaning all neurons in a given layer perform exactly the same computation and receive identical gradients during backpropagation. As a result the model behaves like it has just one effective unit.

👉 **Try this yourself:** 1. Modify the way the network weights are initialized to a random uniform distribution to give the network some variety. How does the model perform now?

👉 **Try this yourself:** 2. Are two perceptrons enough to represent this target function? Will increasing the number of Perceptrons improve the performance?

👉 **Try this yourself:** 3. There might still be a large number of neurons which aren't activating? How does the data distribution impact whether a given Perceptron is activated or not? How is the dataset scaled in relation to the input distribution? Can we use a different activation function to provide some gradient information?

<h3>2D. Learning to fit MORE complex functions</h3>

So clearly there are several important considerations when designing and training a ReLU network. While the **ReLU activation** gives us the ability to build piecewise linear approximations, the effectiveness of learning still depends heavily on:

- The **basis functions** we choose (i.e., the architecture and activation function),
- The **initialization** of the parameters, and
- The **scaling and structure** of the dataset.

Even though ReLU networks are universal function approximators in theory, in practice they are only as effective as our ability to **optimize their parameters**. Poor initialization (e.g., setting all weights to the same value) can lead to **symmetry** in the network — where every neuron behaves identically and no useful function approximation is learned. Similarly, improper scaling of inputs can push most activations into a "dead zone", especially with ReLU, where gradients vanish and learning stalls.

As we move toward fitting more complex functions — ones with curvature, nonlinearity, or high-frequency variation — we need to ensure that the network is **sufficiently expressive**, but also **well-initialized** and **well-conditioned** to allow optimization to succeed.

In the next activity, we’ll explore how to modify our networks to better approximate more complex target functions and investigate what architectural and training decisions make the biggest impact.

Let's consider another slighty more complicated target function:

\begin{align*}
  f^{*}(x) = \left\{ \mathbb{R} \rightarrow \mathbb{R} \,|\, f(x) = 2.04x^{3} + 1.54x^{2} - 0.54x + 1.3 \right\} \tag{2.3}
\end{align*}

In [ ]:
# define a dataset
f_cubic = lambda x: 2.04 * x**3 + 1.54 * x**2 - 0.54 * x + 1.3
dataset = Dataset(f=f_cubic, a=-2, b=2, N=100)

# visualize the observations
fig, ax = plt.subplots(figsize=(8,3))
ax.scatter(dataset.x, dataset.y, s=6, label="observations")
ax.set_xlim(dataset.x.min(), dataset.x.max())
ax.legend(loc="best")
ax.set_xlabel("x_meas")
ax.set_ylabel("y_obs")

In [ ]:
class Perceptron(nn.Module):
    """
    f(x) = ReLU(Wx + B)
    """
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1) # <-- torch uses `kaiming` init for us
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.act(self.linear(x))


class PerceptronLayer(nn.Module):
    def __init__(self, N: int = 1):
        super().__init__()
        self.perceptrons = nn.ModuleList([Perceptron() for _ in range(N)])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_pred = 0
        for perceptron in self.perceptrons:
            y_pred = y_pred + perceptron(x)
        return y_pred

In [ ]:
model = PerceptronLayer(N=2) # <-- lets define 2 perceptrons initially
dataset = Dataset(f=f_cubic, a=-2, b=2, N=100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

plot_loss_curve(losses)
plot_predictions(model, dataset)

<h3>Section 2E. Linear Combinations of Perceptrons</h3>

ReLU functions are **piecewise linear and convex**. As a result, a single-layer ReLU network (a shallow network) constructs its output by summing **convex, piecewise linear components**. This makes it inherently biased toward representing functions that are **convex or composed of sharp transitions** — but not naturally suited to expressing smooth **concave** curves like quadratics or sines.

To model concavity using only ReLU units, the network must carefully combine many "bent" ReLU functions with precisely tuned weights and biases — a process that requires **many units**, and is **difficult to optimize effectively**.

To provide more flexibility to our network, we can introduce a **linear output (readout) layer**. This allows the model to form **both convex and concave** outputs by combining the activations of ReLU units using **both positive and negative weights**. Instead of relying on each ReLU to fit a piece of the function directly, the network can now learn a richer combination of features — dramatically increasing its expressivity, even in shallow architectures.

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, act: bool = True):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.act = nn.ReLU() if act else lambda x: x

    def forward(self, x):
        x = self.act(self.fc(x))
        return x


class PerceptronLayer(nn.Module):
    def __init__(self, N: int = 1):
        super(PerceptronLayer, self).__init__()
        modules = [Perceptron(1, N)]
        modules.append(Perceptron(N, 1, act=False))
        self.layers = nn.ModuleList(modules)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = PerceptronLayer(N=2) # <-- lets define 2 perceptrons initially
dataset = Dataset(f=f_cubic, a=-2, b=2, N=100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100) # <-- does the loss look like it could go lower, should we train for longer?

plot_loss_curve(losses)
plot_predictions(model, dataset)

This architectural tweak — adding a linear combination on top of non-linear activations — is a simple way to overcome the limitations imposed by ReLU's inherent convexity.

<h3>Section 2E. Multi Layer Perceptron Model</h3>

Clearly, making this adjustment provides significantly more expressive power to our network, allowing it to more easily represent complex shapes — including those with both convex and concave regions.

We can further enhance this expressivity by introducing **multiple layers**. Each additional layer allows the network to learn a **piecewise linear approximation of the previous layer’s piecewise linear approximation**. In effect, this means the network can begin to capture **nonlinear interactions and curvature** through the **composition of simple functions**.

In [ ]:
class Perceptron(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, act: bool = True):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.act = nn.ReLU() if act else lambda x: x

    def forward(self, x):
        x = self.act(self.fc(x))
        return x


class MultiLayerPerceptron(nn.Module):
    def __init__(self, hidden_layers: int = 0, hidden_dim: int = 1):
        super(MultiLayerPerceptron, self).__init__()

        # Create input layer
        modules = [Perceptron(1, hidden_dim)]

        # Create hidden layers
        for _ in range(hidden_layers):
            modules.append(Perceptron(hidden_dim, hidden_dim))

        # Create output layer
        modules.append(Perceptron(hidden_dim, 1, act=False)) # just perform linear aggregation and no non-linearity

        # Create the model
        self.layers = nn.ModuleList(modules)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
model = MultiLayerPerceptron(hidden_layers=0, hidden_dim=2) # <-- lets define 2 perceptrons initially
dataset = Dataset(f=f_cubic, a=-2, b=2, N=100)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # <-- how do different optimizers impact performance
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

plot_loss_curve(losses)
plot_predictions(model, dataset)

This compositional structure enables deeper networks to fit functions like **cubics, smooth oscillations, or piecewise-polynomial curves** — tasks that would otherwise require an impractically large number of units in a shallow network. Instead of needing to directly “draw” every bump or bend with individual neurons, the network learns reusable transformations that **build up complexity progressively**.

> In short: a linear output layer gives us flexibility in combining features, and depth gives us the capacity to build increasingly rich representations.

<h3>An aside</h3>

It's also interesting to note that as we make our model larger, it becomes more capable of fitting complex data distributions. Bigger models have more representational capacity, which allows them to capture subtle variations in the dataset that smaller models might miss.

This relates closely to the **Lottery Ticket Hypothesis**, which suggests that within large networks, there often exist smaller, well-initialized sub-networks — "winning tickets" — that are capable of learning the task effectively. As we increase the size of the model, the **probability of finding such a sub-network increases**, even if only a portion of the network is initially well-suited to the task.

Counterintuitively, **larger models can also make optimization easier**. In low-dimensional parameter spaces, local minima can trap optimization. But in higher-dimensional spaces, those same points often become **saddle points**, allowing the optimizer to escape and continue improving. The increased redundancy and flexibility of large models can smooth out the loss landscape, providing **multiple paths to good solutions**.

However, this advantage only materializes if the **optimization procedure is robust enough** to effectively explore the high-dimensional parameter space.

> In short: bigger models aren't just more expressive — they can also be *easier to train*, provided the optimizer is equipped to navigate the landscape.

<h2>Section 3. Putting It Into Practice!</h2>

We're going to break into groups and explore both sides of the learning process: **the data** and **the model**. Your task is to create a challenging function - one that’s complex, nonlinear, and difficult to approximate - potentially even discontinuous - and then build a model that can learn to fit it. Then we're going to try and see just how good this approximation is.

<h3>Section 3A. Fit Your Own Function</h3>

1. **Design a Complex Function**  
   Work together to create a synthetic function with as much structure, curvature, or discontinuity as you'd like. The goal is to make the function *as contorted and expressive as possible* — think beyond simple polynomials!

2. **Generate Sample Data**  
   Sample a set of input-output pairs from your function. You can optionally add noise to simulate real-world data.

3. **Build a Model to Fit It**  
   Use your understanding of model capacity, activation functions, and architecture to design a neural network that can approximate your function.  

4. **Train & Evaluate**  
   Try fitting your model to your data using what you've learned so far. Plot the results and report back as a group on your finding.

In [ ]:
# define a dataset
f_yours = lambda x: ... # <-- go wild!
dataset = Dataset(f=f_yours, ...)

# visualize the observations
fig, ax = plt.subplots(figsize=(8,3))
ax.scatter(dataset.x, dataset.y, s=6, label="observations")
ax.set_xlim(dataset.x.min(), dataset.x.max())
ax.legend(loc="best")
ax.set_xlabel("x_meas")
ax.set_ylabel("y_obs")

In [ ]:
model = ... # <-- define your model here
dataset = ... # <-- define your dataset here
optimizer = ... # <-- define your optimizer here
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, ...)

plot_loss_curve(losses)
plot_predictions(model, dataset)

Let's reconvene and share with the class.

<h3>Section 3B. How well can your model interpolate?</h3>

Neural networks, particularly with ReLU or smooth activations, tend to learn **smooth functions** that fit the data. Given enough data points that sufficiently cover the input space (i.e., fine resolution across the data manifold), they can interpolate well between samples — essentially “connecting the dots” in a reasonable way. This smoothness bias helps the model generalize **within** the distribution — but it doesn’t help beyond it.

In [ ]:
# define your domain
dataset = Dataset(f=f_yours, ...)

# lets erase some of the data
x_min, x_max = -0.1, 0.1
keep_idxs = (dataset.x < x_min) | (dataset.x > x_max)
dataset.x = dataset.x[keep_idxs].unsqueeze(-1)
dataset.y = dataset.y[keep_idxs].unsqueeze(-1)

# visualize the observations
fig, ax = plt.subplots(figsize=(8,3))
ax.scatter(dataset.x, dataset.y, s=6, label="observations")
ax.set_xlim(dataset.x.min(), dataset.x.max())
ax.legend(loc="best")
ax.set_xlabel("x_meas")
ax.set_ylabel("y_obs")

In [ ]:
model = MultiLayerPerceptron(hidden_layers=1, hidden_dim=20) # <-- use the same parameters as before
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 1000)

plot_loss_curve(losses)

In [ ]:
# intra-disitribution loss
dataset = Dataset(f=f_yours, a=-2, b=2, N=100)
mask = (dataset.x < x_min) | (dataset.x > x_max)
dataset.x = dataset.x[mask].unsqueeze(-1)
dataset.y = dataset.y[mask].unsqueeze(-1)
ax = plot_predictions(model, dataset)
print(f"in-distribution loss: {loss_fn(model(dataset.x), dataset.y):.3f}")

# out-of-distribution loss
dataset = Dataset(f=f_yours, a=-2, b=2, N=100)
mask = (dataset.x > x_min) & (dataset.x < x_max)
dataset.x = dataset.x[mask].unsqueeze(-1)
dataset.y = dataset.y[mask].unsqueeze(-1)
ax = plot_predictions(model, dataset, ax)
print(f"out-of-distribution loss: {loss_fn(model(dataset.x), dataset.y):.3f}")
ax.set_xlim(-2, 2)

Understanding where your model performs poorly provides insight into where the dataset has poorly constrained the model and thus where to gather more data. This discussion on how sufficiently and accurately we resolve the data manifold is a crucial aspect of machine learning - in approximation theory e.g. polynomial interpolation, error bounds of the function approximation often depend on the maximum spacing between samples:

\begin{align*}
    \epsilon \propto \delta^{k} \cdot || f^{(k)} ||_{\infty}, \quad
    \delta = max_{i}(\mathcal{X}_{i+1} - \mathcal{X}_{i}) \tag{3.1}
\end{align*}

where:
- $\epsilon$ is the error bound
- $f^{(k)}$ is the $k$-th derivative of a function $f$
- $\delta$ is the largest distance between points

Where a functions value changes rapidly (high $f^{(k)}$) to make sure we can constrain our function (minimize $\epsilon$) we need to sample the points more densely around this area of change (minimize $\delta$). Interpolation is where neural networks shine — but only when your data gives them something to work with.

<h3>Section 3C. How well can your model extrapolate?</h3>

Now it's time to evaluate not just how well it fits the **training data**, but how well it generalizes. So far, your model has only seen inputs within a certain range $\mathcal{D}$ — the domain of your sampled data. But real-world models often need to make predictions on inputs **outside this range**, to generalize to unseen cases or to be robust to adversarial examples.

Let's see if this is the case.




In [ ]:
# in-distribution
id_dataset = Dataset(f=f_yours, ...) # <-- define your in-distribution data
ax = plot_predictions(model, id_dataset)

In [ ]:
# out-of-distribution data
ood_dataset = Dataset(f=f_yours, ...) # <-- in-distribution data + out-of-distribution data i.e. bigger domain
ax = plot_predictions(model, ood_dataset)

Typically the capacity of the networks are leveraged where the data is well defined to force the geometry of the networks activations to fit the geometry of the data manifold.

Extrapolation is difficult because the model receives **no supervision or constraints** in regions outside the training distribution. Unless the network has explicitly learned the **underlying rules** or structure of the data (e.g., physical laws, symmetries, or inductive priors), it has no reason to behave sensibly in unfamiliar regions.

In most practical settings, the model is just an **approximation**: it captures patterns in the data, but not the principles that generate them. As a result, its behavior outside the training domain is often **unconstrained and unpredictable**.

Understanding where your model performs poorly — especially on out-of-distribution (OOD) inputs — gives valuable feedback about **which parts of the input space are under-constrained**. If you believe the model has enough capacity to approximate the target function, poor generalization usually means you need to:

- **Collect more data** in those regions,
- **Resample** with better coverage,
- Or **introduce structure** into the model (e.g., domain knowledge, architectural bias).

> Good generalization is not just about model size — it’s about the match between your data distribution and the problem you’re trying to solve.


In [ ]:
model = MultiLayerPerceptron(hidden_layers=1, hidden_dim=10) # <-- use the same parameters as before
dataset = ood_dataset
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model, losses = training_loop(model, optimizer, loss_fn, dataset.x, dataset.y, 100)

plot_loss_curve(losses)
plot_predictions(model, dataset)

Neural networks are generally good at **interpolating** within the range of their training data — but they often struggle to **extrapolate** beyond it. This is a fundamental limitation, rooted in how these models learn.

<h2>Summary</h2>

1. **Learning as Function Approximation**: Machine learning is about learning a function that maps inputs to outputs using data, assumptions, and optimization.
2. **Hypothesis Spaces & Universal Approximators**: Neural networks are powerful because they can approximate a wide range of functions — given enough capacity and the right architecture.
3. **Practical Exploration**: A model's ability to learn depends not just on the architecture, but on how it's initialized, constrained, and trained — and how well the data represents the task.
4. **Interpolation vs Extrapolation**: Neural networks generalize well within the training range (interpolation), but fail unpredictably outside it (extrapolation).
5. **Final Takeaway**: Building effective models means balancing expressivity, data coverage, and optimization — and always testing their limits.

In the next workshop we'll explore applying neural networks to higher dimensional inputs and solving real-world problems, with a focus on understanding the limitations of these systems and building up intuition about how to address them.